In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./datasets/housing.csv')
df2 = pd.read_csv('./datasets/student-mat.csv')

# Index and Select
- iloc[] to access elements by integer index
- loc[] to access elements by the index of the data frame

#### Columns

In [3]:
# Select columns 0
new_df = df['latitude']

# Select columns 1
new_df = df[['longitude','latitude','population', 'ocean_proximity']]

# Select columns 2
new_df = df.loc[:, ['longitude','latitude','population', 'ocean_proximity']]

# Select columns 3
new_df = df.iloc[:, [0,2,4]]

#### Rows

In [4]:
# Select rows 0
new_df = df.loc[2]

# Select rows 1
new_df = df.loc[[0,2,5,8], :]

# Select rows 2
new_df = df.loc[3:7, :]

# Select rows 3
new_df = df.iloc[3:7, :]

#### Rows and cols

In [5]:
# Select rows-cols 1
new_df = df.loc[[0,2,5,8], ['longitude','latitude','population', 'ocean_proximity']]

# Select rows-cols 2
new_df = df.loc[0:5, ['longitude','latitude','population', 'ocean_proximity']]

# Filter

#### Unique

In [6]:
# Find all distinct values
unique_values = df['ocean_proximity'].unique()
print(unique_values)

['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']


#### Condition filter

In [7]:
print(df.shape)

(20640, 10)


In [8]:
condition = (df['ocean_proximity'] == 'NEAR OCEAN')
filtered_df = df[condition]
print(filtered_df.shape)


condition = df['population'] < 10000
filtered_df = df[condition]
print(filtered_df.shape)


condition = df['ocean_proximity'] != 'INLAND'
filtered_df = df[condition]
print(filtered_df.shape)


condition = (df['population'] > 15000) & (df['households'] < 5000)
filtered_df = df[condition]
print(filtered_df.shape)


condition = df['ocean_proximity'].isin(['NEAR OCEAN', 'NEAR BAY'])
filtered_df = df[condition]
print(filtered_df.shape)

(2658, 10)
(20617, 10)
(14089, 10)
(2, 10)
(4948, 10)


# Function

In [9]:
# Function
def convert(value):
    return value * 10000
df['converted_median_income'] = df['median_income'].apply(convert)

# Lambda
df['converted_median_income'] = df['median_income'].apply(lambda x: x*10000)

In [10]:
def convert_categories(value):
    if value > 10:
        return 'high-incomes'
    elif value > 2 and value < 10:
        return 'moderate-incomes'
    else:
        return 'low-incomes'
df['categorical_median_income'] = df['median_income'].apply(convert_categories)

# Aggregate
**Note**: After groupby, grouped columns are set as index
+ reset index by: `.reset_index()` 

In [11]:
summmation = df['households'].sum()
minimum = df['median_income'].min()
maximum = df['median_house_value'].max()
avg = df['population'].mean()

#### Group by 1 variable
<img src="./assets/1.svg" width="600"/>

In [12]:
sums = df[['population','ocean_proximity']] \
    .groupby('ocean_proximity') \
    .sum()

sums

,population
ocean_proximity,
<1H OCEAN,13889374.0
INLAND,9112744.0
ISLAND,3340.0
NEAR BAY,2817427.0
NEAR OCEAN,3598955.0


In [13]:
# multi col agg
sums = df \
    .groupby('ocean_proximity')[['population', 'ocean_proximity', 'households']] \
    .agg({
        'population': np.sum,
        'households': np.mean})
sums

population              households            
                 population households   population  households
ocean_proximity                                                
<1H OCEAN        13889374.0  4730118.0  1520.290499  517.744965
INLAND            9112744.0  3127759.0  1391.046252  477.447565
ISLAND               3340.0     1383.0   668.000000  276.600000
NEAR BAY          2817427.0  1118931.0  1230.317467  488.616157
NEAR OCEAN        3598955.0  1332308.0  1354.008653  501.244545

#### Group by 2 variables

In [14]:
avgs = df2[['school','sex','G3']] \
    .groupby(['school','sex']) \
    .mean()

avgs

G3
school sex           
GP     F     9.972678
       M    11.060241
MS     F     9.920000
       M     9.761905

#### Count using Groupby then sort

In [15]:
# Insert new column
df.insert(loc=0, column='Count', value=1)

In [16]:
# Agg
df[['ocean_proximity','Count', ]] \
    .groupby('ocean_proximity') \
    .sum() \
    .sort_values('Count', ascending=False)

,Count
ocean_proximity,
<1H OCEAN,9136
INLAND,6551
NEAR OCEAN,2658
NEAR BAY,2290
ISLAND,5


In [17]:
# Agg - reset index
df[['ocean_proximity','Count', ]] \
    .groupby('ocean_proximity') \
    .sum() \
    .sort_values('Count', ascending=False) \
    .reset_index()

,ocean_proximity,Count
0,<1H OCEAN,9136
1,INLAND,6551
2,NEAR OCEAN,2658
3,NEAR BAY,2290
4,ISLAND,5


#### Count using Groupby then sort - Group by 2 variables

In [18]:
# Insert new column
df2.insert(loc=0, column='Count', value=1)

# Agg
df2[['school','sex','Count']] \
    .groupby(['school','sex']) \
    .sum() \
    .sort_values('Count', ascending=False)

Count
school sex       
GP     F      183
       M      166
MS     F       25
       M       21

#### sort_values
```python
# Sort multiple columns: 1st prior = 1st column
df = df.sort_values(["Budget", "Profit"], ascending=[True, False])
```

#### Pivot Tables
- Pivot tables = a summary of the whole data
    + Reorganize the desired data in a different format
    + Transform data from columns to rows or rows to columns
    + Group data by any attribute
    + Include statistics: sum, mean, maximum, minimum, etc.

- Use cases
    + Group data for business problems
        + Eg: Calculating sales by region or products
    + Compare different classes of a data field
        + Eg: Comparing data for males and females
    + Detect unique values in a field
        + Eg: Finding out different types of products
    + Summarize complex tables

In [19]:
df2.pivot_table(
    index=['school','sex'],
    values=['G3','Walc','Dalc','studytime'],
    aggfunc = 'mean')

Dalc         G3      Walc  studytime
school sex                                          
GP     F    1.251366   9.972678  1.956284   2.300546
       M    1.656627  11.060241  2.596386   1.801205
MS     F    1.280000   9.920000  1.960000   2.120000
       M    2.333333   9.761905  3.190476   1.476190

In [20]:
df2.pivot_table(
    index=['school','sex'],
    values=['G3','Walc','Dalc','studytime'],
    aggfunc = {
        'Walc': np.max,
        'studytime': np.mean})

Walc  studytime
school sex                 
GP     F       5   2.300546
       M       5   1.801205
MS     F       4   2.120000
       M       5   1.476190